In [8]:
import pandas as pd
import sklearn
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np
import signal_processing as sigpro
#from pysptools.spectro import FeaturesConvexHullQuotient, SpectrumConvexHullQuotient

from ipywidgets import interact
import time

In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
# try to drop class 1
drop_class_1 = False

df = pd.read_csv('merged_data_with_labels.csv')
header_cols =  ['cm_ree_labels', 'Mineral_Name', 'dataset', 'ID']
columns = header_cols + [str(x) for x in np.arange(350, 2500)]
df = df[columns]

if drop_class_1:
    df = df[df['cm_ree_labels'] != 1]
    df = df.reset_index(drop=True)

/tmp/ipykernel_128561/3148397356.py:4: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('merged_data_with_labels.csv')


In [47]:
# Some clean up:
df['cm_ree_labels'].iloc[[272, 299, 220]] = 2

In [49]:
# Start a timer to see how long this takes takes around 38 seconds on my machine.
start_time = time.time()

wvl = df.columns[len(header_cols):].values.astype('float')  # Choose the columns of holding the data
start_wvl = 100 # start at wavelenght 450
end_wvl = None # stop at this wavelength

ncrs_stack = []
for i, row in df.iloc[:,:].iterrows():
    class_idx = int(row['cm_ree_labels'])
    row = df.iloc[i, len(header_cols):]
    w, data  = wvl[start_wvl:end_wvl], row[start_wvl:end_wvl]
    
    # Calculate Hull correction  ---- this step can take a long time.
    wnew, ynew, whull, yhull = sigpro.convex_hull_removal(w, data)
    
    # Remove Hull Background
    yn = sigpro.get_continuum_removed_spectrum(w, data, ynew)
    
    # Add tp stack
    ncrs_stack.append(yn)

ncrs_stack = np.array(ncrs_stack)

print('elapsed_time:', time.time() - start_time)

elapsed_time: 38.255513191223145


# Raw spectra

In [119]:
# Do plot

colour_idx = {0:'black', 1:'green', 2:'red'}
critical_wvs = [517.0, 580.0, 746.0, 800.0, 867.0]

plt.figure(figsize=(16, 8))
for row, class_idx in zip(df[df.columns[len(header_cols):]].values, df['cm_ree_labels']):
    plt.plot(wvl[start_wvl: end_wvl], row[start_wvl: end_wvl:] - 1.0 * class_idx, 
             c=colour_idx[class_idx], alpha=0.05, zorder=1000)
    _ = plt.xticks(np.arange(wvl.min(), wvl.max(), 100))

for keyline in critical_wvs:
    plt.axvline(keyline, c='gray', alpha=0.25)
    plt.text(keyline, 0.90, s=str(int(keyline)), ha='center', va='bottom', color='gray', 
            bbox={'facecolor':'white', 'edgecolor':'white'})
    
plt.xlabel('wavelength (nm)')
plt.ylabel('corrected absorbtion')
_ =plt.ylim(-2.0, 1.0)

In [155]:
@interact
def highlight_curve(num=(1, len(df))):
    
    colour_idx = {0:'black', 1:'green', 2:'red'}
    critical_wvs = [517.0, 580.0, 746.0, 800.0, 867.0]

    
    plt.figure(figsize=(16, 8))
    for row, class_idx in zip(df[df.columns[len(header_cols):]].values, df['cm_ree_labels']):
        plt.plot(wvl[start_wvl: end_wvl], row[start_wvl: end_wvl:] - 1.2 * class_idx, 
                 c=colour_idx[class_idx], alpha=0.05, zorder=1000)
        _ = plt.xticks(np.arange(wvl.min(), wvl.max(), 100))

    for keyline in critical_wvs:
        plt.axvline(keyline, c='gray', alpha=0.25)
        plt.text(keyline, 0.90, s=str(int(keyline)), ha='center', va='bottom', color='gray', 
               bbox={'facecolor':'white', 'edgecolor':'white'})

    plt.xlabel('wavelength (nm)')
    plt.ylabel('corrected absorbtion')
    
    dataset, name, samp_id = df.iloc[num]["dataset"], df.iloc[num]["Mineral_Name"], df.iloc[num]["ID"]
    class_idx = int(df.iloc[num]['cm_ree_labels'])
    data = row[start_wvl: end_wvl:]
    
    
    plt.plot(wvl[start_wvl: end_wvl], data - 1.2 * class_idx, c='white', alpha=0.9, zorder=999, lw=4.5)
    plt.plot(wvl[start_wvl: end_wvl], data - 1.2 * class_idx, c='blue', alpha=1, lw=2, zorder=1000)
    
    #plt.plot(wvl[start_wvl: end_wvl], data - 1.2 * class_idx, c=colour_idx[class_idx], alpha=1, zorder=1000)
    plt.text(450, 1.1 - 1.2 * class_idx, s=dataset + ' ' + name + ' ' + str(samp_id),  c=colour_idx[class_idx])
    _ =plt.ylim(-3.5, 1.5)

interactive(children=(IntSlider(value=328, description='num', max=656, min=1), Output()), _dom_classes=('widge…

# Hull Corrected Spectra

In [154]:
# Do plot

@interact
def highlight_curve(num=(1, len(df))):
    colour_idx = {0:'black', 1:'green', 2:'red'}
    critical_wvs = [517.0, 580.0, 746.0, 800.0, 867.0]

    plt.figure(figsize=(14, 10))
    for row, class_idx in zip(ncrs_stack, df['cm_ree_labels']):
        plt.plot(wvl[start_wvl: end_wvl], row - 1.0 * class_idx, 
                 c=colour_idx[class_idx], alpha=0.05, zorder=100)
        _ = plt.xticks(np.arange(wvl.min(), wvl.max(), 100))

    for keyline in critical_wvs:
        plt.axvline(keyline, c='b', alpha=0.25)
        plt.text(keyline, 0.90, s=str(int(keyline)), ha='center', va='bottom', color='b', 
                bbox={'facecolor':'white', 'edgecolor':'white'})

    plt.xlabel('wavelength (nm)')
    plt.ylabel('corrected absorbtion')
    #_ =plt.ylim(-2.0, 1.0)
    
    this_row = df.iloc[num]
    dataset, name, samp_id = this_row["dataset"], this_row["Mineral_Name"], this_row["ID"]
    class_idx = int(this_row['cm_ree_labels'])
    w = wvl[start_wvl:end_wvl]
    spectra = this_row[len(header_cols):][start_wvl:end_wvl]
    
    # Calculate Hull correction  ---- this step can take a long time.
    wnew, ynew, whull, yhull = sigpro.convex_hull_removal(w, spectra)
    
    # Remove Hull Background
    yn = sigpro.get_continuum_removed_spectrum(w, spectra, ynew)
    
    plt.plot(w, yn - 1.0 * class_idx, c='white', alpha=0.9, zorder=999, lw=4.5)
    plt.plot(w, yn - 1.0 * class_idx, c='blue', alpha=1, lw=2, zorder=1000)

    plt.text(450, 1.1 - 1.0 * class_idx, s=dataset + ' ' + name + ' ' + str(samp_id),  c=colour_idx[class_idx])
    _ =plt.ylim(-2.0, 1.5)

interactive(children=(IntSlider(value=328, description='num', max=656, min=1), Output()), _dom_classes=('widge…

# Select a subset of the wavelengths in the spectra

In [190]:
ree_wvl_range = slice(100, 650, )  # 450 - 999 nm
X = ncrs_stack[:, ree_wvl_range] # features
y = df['cm_ree_labels'].values
print(X.shape, y.shape)

(656, 550) (656,)


# Do min / max scaler within the 450 to 900 nm range

In [211]:
def min_max_scaler(arr):
    scaler = MinMaxScaler()
    return scaler.fit_transform(arr.reshape(-1, 1))

Xmm = np.apply_along_axis(min_max_scaler, axis=1, arr=X)[..., 0]
Xmm.shape

(656, 550)

In [212]:
@interact
def highlight_curve(num=(1, len(df))):
    
    plt.figure(figsize=(16, 8))
    for i, row in enumerate(Xmm):
        class_idx = int(df['cm_ree_labels'].iloc[i])
        plt.plot(wvl[ree_wvl_range], row - 1.5 * class_idx, c=colour_idx[class_idx], alpha=0.05, zorder=1000)
    _ = plt.xticks(np.arange(wvl.min(), wvl.max(), 100))

    for keyline in critical_wvs:
        plt.axvline(keyline, c='b', alpha=0.25)
        plt.text(keyline, 0.90, s=str(int(keyline)), ha='center', va='bottom', color='b', 
                bbox={'facecolor':'white', 'edgecolor':'white'})

    plt.xlabel('wavelength (nm)')
    plt.ylabel('corrected absorbtion')
    _ =plt.ylim(-3.5, 1.5)
    
    dataset, name, samp_id = df.iloc[num]["dataset"], df.iloc[num]["Mineral_Name"], df.iloc[num]["ID"]
    class_idx = int(df.iloc[num]['cm_ree_labels'])
    plt.plot(wvl[ree_wvl_range], Xmm[num] - 1.5 * class_idx, c=colour_idx[class_idx], alpha=1, zorder=1000)
    plt.text(450, 1.1 - 1.5 * class_idx, s=dataset + ' ' + name + ' ' + samp_id,  c=colour_idx[class_idx])
    _ =plt.ylim(-3.5, 1.5)

interactive(children=(IntSlider(value=328, description='num', max=656, min=1), Output()), _dom_classes=('widge…

In [213]:
@interact
def quick_sig_plot(n=(1, len(df))):
    

    plt.figure(figsize=(12, 8))
    for i, s in enumerate(ncrs_stack[:, ree_wvl_range]):
        class_idx = int(df['cm_ree_labels'].iloc[i])
        plt.plot(wvl[ree_wvl_range], min_max_scaler(s.reshape(-1, 1)),  c=colour_idx[class_idx], alpha=0.02)
    
    dataset, name, samp_id = df["dataset"].iloc[i], df["Mineral_Name"].iloc[i], df["ID"].iloc[i]    
    class_idx = int(df['cm_ree_labels'].iloc[n])

    sig = ncrs_stack[n, ree_wvl_range]
    plt.plot(wvl[ree_wvl_range], sig, c=colour_idx[class_idx], linestyle='dotted',)
    plt.plot(wvl[ree_wvl_range], min_max_scaler(sig.reshape(-1, 1)),  c=colour_idx[class_idx])
    plt.text(wvl[ree_wvl_range].max(), 1.0, 
             s=f'         {dataset}   {name}   {samp_id}', size=14, c=colour_idx[class_idx])
    plt.ylim(-0.1,1.1)
    plt.xlabel('wavelength (nm)')
    
    

interactive(children=(IntSlider(value=328, description='n', max=656, min=1), Output()), _dom_classes=('widget-…

# Recasting X as Xmm

In [214]:
#X = Xmm
X

450       451       452       453       454       455       456  \
0    0.998643  0.998947  0.999169  0.999387  0.999605  0.999806   0.99992   
1         1.0  0.999979  0.999935  0.999926  0.999945       1.0  0.999999   
2    0.998801  0.998933  0.999161   0.99938   0.99957  0.999773  0.999898   
3     0.99999       1.0  0.999967  0.999969       1.0       1.0       1.0   
4         1.0       1.0       1.0       1.0       1.0       1.0       1.0   
..        ...       ...       ...       ...       ...       ...       ...   
651       1.0  0.999992       1.0  0.999992  0.999958       1.0   0.99997   
652       1.0   0.99996  0.999935  0.999924  0.999916  0.999845  0.999851   
653       1.0       1.0       1.0       1.0  0.999992  0.999935  0.999943   
654  0.999952       1.0       1.0       1.0  0.999999  0.999951  0.999913   
655  0.946313  0.946127  0.945992  0.945857  0.945746  0.945765  0.945804   

          457       458       459  ...       990       991       992  \
0    0.999983       1.0       1.0  ...  0.963092  0.963561   0.96403   
1    0.999987  0.999999       1.0  ...  0.971573  0.971953  0.972356   
2    0.999972       1.0       1.0  ...  0.967603  0.968068  0.968509   
3         1.0       1.0       1.0  ...  0.899069  0.899822  0.900556   
4         1.0       1.0       1.0  ...  0.907181  0.907874  0.908537   
..        ...       ...       ...  ...       ...       ...       ...   
651  0.999921  0.999893       1.0  ...   0.97842  0.978388  0.978358   
652  0.999866  0.999862  0.999783  ...  0.974276  0.974251  0.974257   
653  0.999976       1.0  0.999952  ...  0.929084  0.928951  0.928836   
654  0.999898    0.9999   0.99988  ...  0.952191  0.952152   0.95211   
655  0.945854  0.945922  0.946067  ...  0.990992  0.990808  0.990637   

          993       994       995       996       997       998       999  
0    0.964509  0.964992  0.965466  0.965944  0.966411  0.966859   0.96731  
1     0.97275  0.973169  0.973589  0.973997  0.974391  0.974782   0.97516  
2    0.968955  0.969394  0.969833  0.970285  0.970738  0.971199  0.971649  
3    0.901296  0.902015  0.902734  0.903468  0.904202  0.904937  0.905683  
4     0.90921  0.909891  0.910566  0.911253   0.91194   0.91262  0.913306  
..        ...       ...       ...       ...       ...       ...       ...  
651  0.978329  0.978309  0.978287  0.978249  0.978224  0.978211  0.978201  
652  0.974254  0.974246  0.974255  0.974266  0.974276  0.974296  0.974346  
653  0.928727  0.928626  0.928532  0.928441  0.928353   0.92827  0.928204  
654  0.952085  0.952074  0.952057  0.952056  0.952043  0.951997  0.951961  
655  0.990464  0.990287  0.990115  0.989934  0.989757  0.989595  0.989423  

[656 rows x 550 columns]

In [215]:
y = df['cm_ree_labels']
y.shape, X.shape

((656,), (656, 550))

In [244]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier()

clf.fit(Xmm, y)

print(clf.score(X, y))

0.6570121951219512


In [272]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=40, min_samples_leaf=2)

clf.fit(Xmm, y)

print(clf.score(Xmm, y))

0.8384146341463414


# Get the N most import features

In [273]:
N = 10
imp = clf.feature_importances_
sorted_index_array = np.argsort(imp)[::-1]  # sort largest to smallest
import_wvls = wvl[ree_wvl_range][sorted_index_array][:N]
import_wvls

array([866., 698., 864., 867., 860., 865., 863., 856., 476., 696.])

In [277]:
def plot_important_feats(N=2, alpha=1.0):
    imp = clf.feature_importances_
    sorted_index_array = np.argsort(imp)[::-1][:N]  # sort largest to smallest
    import_wvls = wvl[ree_wvl_range][sorted_index_array]
    #print(sorted_index_array)
    #print(import_wvls)
    
    fig, ax = plt.subplots(figsize=(16, 8))
    for i, row in enumerate(Xmm.values):
        class_idx = int(df['cm_ree_labels'].iloc[i])
        ax.plot(wvl[ree_wvl_range], row - 1.5 * class_idx, c=colour_idx[class_idx], alpha=0.05, zorder=1000)
    _ = ax.set_xticks(np.arange(wvl.min(), wvl.max(), 100))

    # feature importance annotation
    cmap = 'plasma'
    viridis = plt.get_cmap(cmap) 
    cnorm = colors.Normalize(vmin=0, vmax=1)
    scalarMap = cm.ScalarMappable(cmap=viridis, norm=cnorm)
    cvalues = np.linspace(1, 0.1, len(import_wvls))
    
    for impwvl, c in zip(import_wvls, cvalues):
        colorVal = scalarMap.to_rgba(c)
        im = ax.axvline(impwvl, alpha=alpha,
                    color=colorVal, lw=2
                    )

    ax.set_xlabel('wavelength (nm)')
    ax.set_ylabel('corrected absorbtion')

    # Create the inset axes and use it for the colorbar.
    cax = fig.add_axes([0.4, 0.25, 0.33, 0.33])
    im = cax.imshow(np.repeat(cvalues.reshape(-1,1), 1, axis=0), origin="lower", cmap=cmap, aspect=40, 
                   vmin=np.amin(cvalues), vmax=1.0)
    cax.set_title('Feature \n importance', size=10)
    #cax.set_ylim(np.amin(cvalues), 1.0)
    cbar = fig.colorbar(im, cax=cax, cmap=cmap, )
    
    _ = ax.set_ylim(-3.5, 1.5)

In [278]:
@interact
def important(N=(1, 100)):
    ind = np.argpartition(imp, -N)[-N:][::-1]
    import_wvls = wvl[ree_wvl_range][ind]
    #print(ind)
    print(import_wvls)

interactive(children=(IntSlider(value=50, description='N', min=1), Output()), _dom_classes=('widget-interact',…

In [279]:
import ipywidgets as ip

ip.interact(plot_important_feats, N=(1, 100, 1), alpha=(0.1, 1.0, 0.1))

interactive(children=(IntSlider(value=2, description='N', min=1), FloatSlider(value=1.0, description='alpha', …

<function __main__.plot_important_feats(N=2, alpha=1.0)>

# Split the data set 

In [280]:
Xmm = pd.DataFrame(Xmm, columns=wvl[ree_wvl_range].astype(int))

In [285]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(Xmm, y, test_size=0.5, random_state=21)

In [289]:
clf = RandomForestClassifier(max_depth=8, min_samples_leaf=2).fit(X_train, y_train)

print(clf.score(X_test, y_test))

0.47560975609756095


In general, you need three datasets for training and evaulating models:

* Train — For fitting a candidate model.
* Val — For comparing different hyperparameter choices (model selection).
* Test — For seeing how well the final model generalizes.

Be careful; people give these different names sometimes. And people do different things with respect to comparing different algorithms — should we re-use the Val dataset, or use Test? If we use Test, are we possibly overfitting to that...

Anyway, we at least need to split the validation set into two parts: one to validate against while selecting hyperparameters (sometimes also called the dev set), and one to assess the likely real-world performance of the trained model.

Note that you should only predict on the test set once, at the end of model selection and tuning.

In [287]:
clf

RandomForestClassifier(max_depth=40, min_samples_leaf=2)


The IID assumption
IID stands for independent and identically distributed, and most machine learning algorithms implicitly assume your data are IID. To put it another way: our samples are random

** We assume the IID assumption is valid so we can split randomly

# Dummy classifier

The dummy classifier tries to answer the question, "How well can I do with zero intelligence?". In other words, given the relative proportions of facies, what would you expect from random weighted guesses?

In [290]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report

dummy = DummyClassifier()

dummy.fit(X_train, y_train)

print(classification_report(y_train, dummy.predict(X_train)))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        89
           1       0.00      0.00      0.00        97
           2       0.43      1.00      0.60       142

    accuracy                           0.43       328
   macro avg       0.14      0.33      0.20       328
weighted avg       0.19      0.43      0.26       328



/home/evan/miniconda3/envs/reflectance/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/evan/miniconda3/envs/reflectance/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/evan/miniconda3/envs/reflectance/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

In [292]:
from sklearn.svm import SVC

model = SVC(C=1.5)
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.5152439024390244

In [293]:
Cs = np.logspace(0.00001, 2, 50)
print(Cs)
y_train_sc = []
y_val_sc = []
for c in Cs:
    model = SVC(C=c, gamma= 1 / 200)
    model.fit(X_train, y_train)
    y_train_sc.append(model.score(X_train, y_train))
    y_val_sc.append(model.score(X_test, y_test))

plt.plot(Cs, y_train_sc, '-o')
plt.plot(Cs, y_val_sc, '-o')

[  1.00002303   1.09856592   1.20681929   1.32574002   1.45637927
   1.5998918    1.75754614   1.93073583   2.12099175   2.32999561
   2.55959483   2.81181891   3.0888973    3.39327918   3.72765503
   4.09498049   4.49850243   4.94178767   5.42875452   5.96370739
   6.55137485   7.19695142   7.90614352   8.68521985   9.54106685
  10.48124955  11.51407844  12.64868293  13.89509206  15.26432313
  16.76847908  18.4208555   20.23605813  22.23013197  24.42070308
  26.82713444  29.47069706  32.37475799  35.5649869   39.06958296
  42.91952412  47.14884088  51.79491717  56.89882072  62.50566611
  68.66501357  75.43130697  82.86435516  91.02986058 100.        ]


In [294]:
Cs[np.argmax(y_val_sc)]

7.196951424206053

In [295]:
C_best = Cs[np.argmax(y_val_sc)]

In [297]:
model = SVC(C=C_best, gamma= 1 / 200)
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.5365853658536586

In [300]:
from sklearn.metrics import classification_report

print(classification_report(y_test, model.predict(X_test), digits=3, ))

              precision    recall  f1-score   support

           0      0.483     0.663     0.559        86
           1      0.822     0.311     0.451       119
           2      0.497     0.667     0.569       123

    accuracy                          0.537       328
   macro avg      0.601     0.547     0.526       328
weighted avg      0.611     0.537     0.524       328



In [301]:
from sklearn.metrics import confusion_matrix
import itertools


def plot_confusion_matrix(y_true, y_pred, normalized=True):
    """
    The sklearn version of this function does not
    allow you to pass in a cross_validation result.
    """
    # Compute confusion matrix
    cnf_matrix = confusion_matrix(y_true, y_pred)
    if normalized:
        norm = (cnf_matrix.T / np.sum(cnf_matrix, axis=1)).T
    else:
        norm = cnf_matrix
    classes = np.unique(y_true)

    # Plot non-normalized confusion matrix.
    plt.figure(figsize=(6, 6))
    plt.imshow(norm, interpolation='nearest', cmap='Greens', vmin=0, vmax=1)
    plt.title("Confusion matrix")
    plt.colorbar(shrink=0.67)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    # Print the support numbers inside the plot.
    thresh = cnf_matrix.max() / 2.
    for i, j in itertools.product(range(cnf_matrix.shape[0]), range(cnf_matrix.shape[1])):
        plt.text(j, i, format(cnf_matrix[i, j], 'd'),
                horizontalalignment="center",
                color="white" if cnf_matrix[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.ylim(2.5, -0.5)  # Bug in mpl.
    plt.tight_layout()
    plt.show()

    return


In [304]:
plot_confusion_matrix(y_test, model.predict(X_test))

### VIS

## Correct classficiations

In [310]:
model.predict(X_test) == y_test

131    False
398    False
142     True
574    False
146     True
       ...  
282     True
280    False
140    False
446     True
467    False
Name: cm_ree_labels, Length: 328, dtype: bool

In [315]:
X_test

450       451       452       453       454       455       456  \
131  1.000000  0.960405  0.995427  1.000000  0.979494  0.994862  1.000000   
398  0.381372  0.386531  0.393350  0.393698  0.393729  0.395771  0.394600   
142  0.000000  0.009101  0.020304  0.033591  0.048440  0.064511  0.082139   
574  1.000000  0.999128  1.000000  1.000000  0.999505  0.999481  1.000000   
146  0.715689  0.715261  0.712434  0.712840  0.714150  0.715220  0.716022   
..        ...       ...       ...       ...       ...       ...       ...   
282  0.742343  0.754723  0.769400  0.783353  0.797790  0.812826  0.823091   
280  0.999848  0.999300  1.000000  0.996479  0.995931  1.000000  0.996782   
140  0.995906  0.998108  0.999831  1.000000  1.000000  1.000000  0.999306   
446  0.816789  0.815422  0.817043  0.819074  0.823280  0.829541  0.830939   
467  0.994213  1.000000  0.992766  1.000000  0.992766  1.000000  0.992766   

          457       458       459  ...       990       991       992  \
131  0.997922  0.992711  0.990674  ...  0.904126  0.911182  0.917786   
398  0.393534  0.392519  0.391475  ...  0.623088  0.627376  0.630210   
142  0.100652  0.119181  0.135388  ...  0.993302  0.991898  0.990623   
574  1.000000  0.998982  0.997602  ...  0.983942  0.985693  0.985847   
146  0.718219  0.721210  0.723193  ...  0.021249  0.019010  0.016356   
..        ...       ...       ...  ...       ...       ...       ...   
282  0.834819  0.847219  0.856148  ...  0.151334  0.153805  0.156142   
280  0.995588  0.995513  0.992283  ...  0.402879  0.404281  0.406820   
140  0.998314  0.996833  0.993594  ...  0.168987  0.149887  0.131721   
446  0.833585  0.837261  0.837914  ...  0.188238  0.171230  0.155477   
467  1.000000  0.992766  1.000000  ...  0.011255  0.008397  0.005539   

          993       994       995       996       997       998       999  
131  0.924833  0.930889  0.933524  0.933305  0.933054  0.929679  0.929469  
398  0.633773  0.637332  0.641007  0.645542  0.650286  0.654608  0.660201  
142  0.989373  0.988125  0.987322  0.987304  0.987141  0.987036  0.987326  
574  0.986190  0.985479  0.985728  0.987874  0.989498  0.992653  0.994160  
146  0.013899  0.011655  0.009303  0.007630  0.005529  0.002513  0.000000  
..        ...       ...       ...       ...       ...       ...       ...  
282  0.158590  0.161016  0.163620  0.166728  0.169743  0.173220  0.175959  
280  0.409556  0.412043  0.414708  0.417618  0.420500  0.423675  0.426612  
140  0.114594  0.098058  0.083055  0.068833  0.056823  0.046181  0.038159  
446  0.141628  0.129185  0.118915  0.109654  0.103844  0.100777  0.101038  
467  0.002681  0.014290  0.011432  0.008574  0.005716  0.002858  0.000000  

[328 rows x 550 columns]

In [375]:
colour_idx = {0:'black', 1:'green', 2:'red'}
correct = y_test == model.predict(X_test)
plt.figure(figsize=(10,8))
for (i, row), label in zip(X_test[correct].iterrows(), y_test[correct]):
    plt.plot(row - 1.2 * label, c=colour_idx[label], alpha=0.1, zorder=1000)

## incorrect classficiations

In [377]:
wrong = y_test == model.predict(X_test)
X_test.index[wrong]

Int64Index([142, 146, 178, 504, 417, 643, 590, 254, 255,  65,
            ...
            217,  92, 252, 396, 460, 615, 157, 654, 282, 446],
           dtype='int64', length=176)

In [388]:
wrong

131    True
398    True
574    True
412    True
638    True
       ... 
201    True
195    True
280    True
140    True
467    True
Name: cm_ree_labels, Length: 152, dtype: bool

In [393]:
y_pred

array([0, 2, 2, 1, 2, 2, 2, 1, 0, 0, 2, 0, 0, 2, 1, 1, 1, 2, 2, 2, 2, 2,
       2, 0, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 2, 0, 0, 2, 2, 2, 0, 2, 0, 1,
       0, 0, 1, 0, 0, 2, 1, 2, 2, 0, 1, 0, 2, 0, 2, 2, 0, 0, 0, 2, 2, 2,
       2, 2, 2, 2, 0, 0, 0, 0, 2, 1, 0, 2, 2, 2, 2, 2, 0, 1, 0, 0, 0, 1,
       0, 0, 2, 2, 0, 0, 0, 2, 2, 1, 2, 0, 0, 0, 2, 2, 2, 2, 1, 2, 2, 2,
       2, 1, 0, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 1, 2, 0, 0, 2, 2, 2, 0, 2,
       0, 0, 1, 2, 0, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 2, 2, 0, 2, 2, 2, 1,
       1, 2, 2, 2, 1, 2, 2, 2, 2, 0, 0, 2, 0, 0, 2, 0, 0, 0, 2, 0, 1, 2,
       0, 2, 2, 2, 2, 0, 2, 0, 1, 2, 2, 0, 2, 2, 2, 0, 0, 0, 2, 2, 0, 2,
       2, 0, 0, 0, 1, 0, 0, 2, 0, 2, 0, 2, 2, 2, 0, 1, 1, 0, 1, 0, 0, 1,
       1, 2, 0, 0, 1, 2, 2, 2, 2, 2, 2, 0, 2, 0, 1, 2, 0, 2, 2, 2, 0, 2,
       2, 1, 0, 0, 0, 2, 1, 2, 0, 0, 2, 0, 0, 2, 2, 0, 2, 2, 2, 2, 0, 2,
       2, 2, 2, 2, 1, 0, 0, 0, 2, 2, 2, 2, 1, 0, 2, 1, 2, 1, 0, 2, 2, 0,
       2, 1, 2, 1, 2, 1, 2, 2, 0, 0, 1, 2, 2, 0, 2,

In [403]:
colour_idx = {0:'black', 1:'green', 2:'red'}
wrong_idx = X_test.index[y_test != model.predict(X_test)]
y_pred = model.predict(X_test)

@interact
def incorrect_plot(n_val=sorted(wrong_idx)):
    plt.figure(figsize=(10,8))
    for (i, row), label, pred in zip(X_test.iterrows(), y_test, y_pred):
        if label != pred:
            plt.plot(row - 1.2 * label, c=colour_idx[label], alpha=0.1, zorder=1000)
    
    
    plt.plot(X_test.loc[n_val] - 1.2 * y_test.loc[n_val], c=colour_idx[y_test[n_val]], alpha=1.0, zorder=1000)
    
    print(df.loc[n_val, header_cols])
    
    this_pred = X_test.loc[n_val].values
    print(this_pred.shape)
    print('\n PREDICTED AS: ', model.predict(this_pred.reshape(1, -1)))

interactive(children=(Dropdown(description='n_val', options=(6, 9, 11, 12, 13, 14, 16, 20, 23, 26, 27, 37, 39,…

# Probabilitistic predictions

In [51]:
from sklearn.model_selection import cross_val_predict
from sklearn.pipeline import make_pipeline

In [52]:
m = make_pipeline(SVC(probability=True))

y_prob = cross_val_predict(m, X_train, y_train,
                           method='predict_proba')

np.set_printoptions(precision=4)
y_prob[:5]

array([[0.3845, 0.6155],
       [0.4434, 0.5566],
       [0.4887, 0.5113],
       [0.6628, 0.3372],
       [0.2553, 0.7447]])